In [1]:
from tsai.models import InceptionTime, TST, ResNet
# from tsai.data.core import get_UCR_data
# from tsai.learner import TSClassifier
import os
import numpy as np
import sys
print(sys.executable)
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path
from preprocessing_modules import create_time_windows_with_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import wandb
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")
from fastai.optimizer import SGD
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback


/home/nwertheim/miniconda3/bin/python


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nwertheim/.netrc
wandb: Currently logged in as: noorwertheim to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
num_epochs = 10

In [3]:
'''MY DATA'''
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2.npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
# test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test_2.npy")
# test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
# print(len(test_data))
print(train_data[0])
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter

# Example dataset with windows and labels
train_windows, train_labels = create_time_windows_with_labels(train_data)
# test_windows, test_labels = create_time_windows_with_labels(test_data)

# Count label distribution
train_label_counts = Counter(train_labels)
# test_label_counts = Counter(test_labels)

# Print the counts
print("Train label distribution:")
print(f"  Term (0): {train_label_counts[0]}")
print(f"  Preterm (1): {train_label_counts[1]}")

# print("\nTest label distribution:")
# print(f"  Term (0): {test_label_counts[0]}")
# print(f"  Preterm (1): {test_label_counts[1]}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
# test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
# test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
# test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

train_labels_tensor = train_labels_tensor.long()
# test_labels_tensor = test_labels_tensor.long()

744
{'record_name': 'Hopper-2024_11_03_14_13_16-0000010179-0002-chan0', 'signal': array([[ 0.70569938],
       [ 1.58579907],
       [ 1.22047813],
       ...,
       [-0.30476434],
       [-0.16021862],
       [-0.40381152]]), 'fs': 20, 'preterm': 1}
windows [[ 0.70569938  1.58579907  1.22047813 ... -0.17193872 -0.14515478
  -0.11458649]
 [-0.08064283 -0.04361066 -0.00404181 ... -0.18969212 -0.23196248
  -0.26956699]
 [-0.3023395  -0.33016298 -0.35305939 ... -1.33257346 -1.29643617
  -1.24306042]
 ...
 [-0.07657955 -0.07240886 -0.06775404 ...  0.12142254  0.09565373
   0.0689264 ]
 [ 0.04192145  0.01471362 -0.01202844 ... -0.02479477 -0.02070454
  -0.01655757]
 [-0.0117706  -0.00703778 -0.0017953  ... -0.05449655 -0.06427623
  -0.07482169]]
labels [1 1 1 ... 0 0 0]
Train label distribution:
  Term (0): 3342
  Preterm (1): 6558


In [4]:
import torch
import optuna
import numpy as np
from tsai.models.ResNet import ResNet
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.losses import CrossEntropyLossFlat
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, Subset
from sklearn.metrics import average_precision_score
from fastai.metrics import Metric
from fastai.callback.wandb import WandbCallback
from fastai.optimizer import SGD, Adam
# Create TensorDataset from your data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# ===== Compute class weights =====
train_labels_np = train_labels_tensor.numpy()
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels_np), y=train_labels_np)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

print(len(train_windows_tensor))
# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]  # Get probabilities for class 1
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    print('Trial:', trial)
    print(f"Starting trial {trial.number}")

    learning_rate = trial.suggest_loguniform('lr', 1e-7, 1e-6)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    optimizer_name = trial.suggest_categorical('optimizer', ['SGD', 'Adam'])

    # Map optimizer name to fastai optimizer function
    opt_func_map = {
    'SGD': SGD,
    'Adam': Adam,
    }

    opt_func = opt_func_map[optimizer_name]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for train_idx, val_idx in kf.split(np.arange(len(dataset))):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

        dls = DataLoaders.from_dsets(
            train_dataset,
            val_dataset,
            bs=batch_size,
            shuffle=True,
            num_workers=0
        )

        model = ResNet(c_in=1, c_out=2).to(device)

        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        learn = Learner(
            dls, model,
            loss_func=loss_func,
            opt_func=opt_func,
            metrics=AveragePrecision(),
            cbs=[
                EarlyStoppingCallback(monitor='valid_loss', patience=3),
            ]
        )

        learn.fit_one_cycle(num_epochs, learning_rate)
        ap = learn.validate()[1]  
        avg_precisions.append(ap)
        

    return np.mean(avg_precisions)

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)


# Save the study
import joblib
joblib.dump(study, "ResNet_tune.pkl")

/home/nwertheim/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-15 22:49:42,060] A new study created in memory with name: no-name-44fe95ba-b930-4c96-8561-de0890c6d3d6


9900
Trial: <optuna.trial._trial.Trial object at 0x14d81055d8b0>
Starting trial 0


epoch,train_loss,valid_loss,avg_precision,time
0,0.695638,0.714094,0.673751,00:10
1,0.696393,0.702747,0.681808,00:09
2,0.693909,0.696603,0.693177,00:09
3,0.690262,0.698148,0.689297,00:09
4,0.693839,0.696534,0.691036,00:09
5,0.688097,0.692809,0.701232,00:09
6,0.686012,0.691988,0.695287,00:09
7,0.689084,0.694431,0.695780,00:09
8,0.687158,0.692581,0.699046,00:09
9,0.686045,0.693887,0.697179,00:09


No improvement since epoch 6: early stopping


epoch,train_loss,valid_loss,avg_precision,time
0,0.707644,0.701972,0.633448,00:14
1,0.697293,0.696957,0.637592,00:15
2,0.692327,0.691658,0.711087,00:09
3,0.689440,0.690966,0.720763,00:09
4,0.690104,0.689691,0.728840,00:09
5,0.687413,0.688983,0.761784,00:09
6,0.688117,0.688560,0.764759,00:09
7,0.690637,0.688686,0.761349,00:09
8,0.690024,0.688784,0.755840,00:09
9,0.687060,0.690365,0.721047,00:09


...

Trial: <optuna.trial._trial.Trial object at 0x14d7f6aa2330>
Starting trial 7


epoch,train_loss,valid_loss,avg_precision,time
0,0.696034,0.693981,0.656293,00:08
1,0.692835,0.691082,0.669883,00:08
2,0.690596,0.689215,0.750776,00:08
3,0.687964,0.688201,0.771409,00:08
4,0.688554,0.688979,0.742719,00:08
5,0.688490,0.686670,0.761479,00:08
6,0.686369,0.686556,0.772862,00:08
7,0.686295,0.687470,0.771601,00:08
8,0.686003,0.688146,0.771831,00:08
9,0.683117,0.686464,0.770784,00:08


epoch,train_loss,valid_loss,avg_precision,time
0,0.726307,0.707576,0.645363,00:08
1,0.709931,0.698066,0.684471,00:08
2,0.700970,0.689320,0.699064,00:08
3,0.690497,0.687435,0.708835,00:08
4,0.686764,0.685634,0.711612,00:08
5,0.690213,0.685606,0.711825,00:08
6,0.687028,0.685826,0.707509,00:08
7,0.688389,0.686216,0.715755,00:08
8,0.689391,0.685788,0.708514,00:08


No improvement since epoch 5: early stopping


epoch,train_loss,valid_loss,avg_precision,time
0,0.707234,0.717122,0.643528,00:08
1,0.703340,0.717739,0.693198,00:08
2,0.695264,0.697324,0.690786,00:08
3,0.690865,0.694724,0.686591,00:08
4,0.686374,0.692549,0.689837,00:08


In [15]:


print(study.best_trial)

FrozenTrial(number=13, state=1, values=[0.813775278468609], datetime_start=datetime.datetime(2025, 5, 15, 20, 10, 33, 836575), datetime_complete=datetime.datetime(2025, 5, 15, 20, 14, 55, 384272), params={'lr': 4.6498121326725904e-07, 'batch_size': 32, 'optimizer': 'Adam'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64)), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=13, value=None)


In [16]:
# import torch
# import optuna
# import numpy as np
# from tsai.models.ResNet import ResNet
# from fastai.learner import Learner
# from fastai.data.core import DataLoaders
# from fastai.metrics import accuracy
# from sklearn.model_selection import KFold
# from torch.utils.data import TensorDataset, Subset
# from fastai.callback.wandb import WandbCallback
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# from fastai.metrics import accuracy
# from fastai.losses import CrossEntropyLossFlat
# from sklearn.model_selection import KFold
# from torch.utils.data import Subset
# from tsai.models.ResNet import ResNet

# # Create TensorDataset from your data
# dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# def objective(trial):
#     print('Trial: ', trial)
#     learning_rate = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     accs = []

#     for train_idx, val_idx in kf.split(np.arange(len(dataset))):
#         train_dataset = Subset(dataset, train_idx)
#         val_dataset = Subset(dataset, val_idx)

#         dls = DataLoaders.from_dsets(
#             train_dataset,
#             val_dataset,
#             bs=batch_size,
#             shuffle=True,
#             num_workers=0  # for compatibility and reproducibility
#         )
#         model = ResNet(c_in=1, c_out=2)
#         learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

#         learn.fit_one_cycle(5, lr_max=learning_rate)
#         acc = learn.validate()[1].item()
#         accs.append(acc)

#     return np.mean(accs)

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=25)

# print("Best trial:")
# print(study.best_trial)

# # Save the study if you want
# import joblib
# joblib.dump(study, "resnet_optuna_study.pkl")
